In [4]:
import numpy as np
import numpy.random as rd
import matplotlib as mpl
import matplotlib.pyplot as mp
import matplotlib.animation as anim
from jupyterthemes import jtplot
# from numba import jit, autojit, vectorize, cuda
import joblib as job

jtplot.style()
mp.switch_backend("TkAgg")
mpl.interactive(True)
mp.rcParams["text.usetex"] = True
mp.rcParams["font.family"] = "serif"
mp.rcParams["figure.figsize"] = (10,10)

#### TSP with algorithm with gpu accel

In [5]:
class TSP(object):
    
    def __init__(self, N, boltz=False):
        rd.seed(42)
        self.T0 = 1/8
        self.N = N
        self.cities = rd.random((N,2))
        self.distances = []
        self.temp = []
        self.T = self.T0
        self.count = 0
        self.clist = []
        self.dlist = []
        self.boltz = boltz
        if boltz:
            self.mode = "Boltzmann"
        else:
            self.mode = "threshold"
        
#     @autojit
    def distance(self, cities):
        d = 0
        city1, city2 = cities, np.roll(cities, -1, axis=0)
        x1,y1 = city1[:,0], city1[:,1]
        x2,y2 = city2[:,0], city2[:,1]
        d = np.hypot((x2-x1), (y2-y1))
        return np.sum(d)
    
#     @autojit
    def get_dist(self):
        return np.sum(np.diff(self.cities))
    
#     @autojit
    def otherroutes(self, cities):
        new_cities = np.copy(cities)
        p = rd.randint(self.N)
        l = rd.randint(self.N//2)
        new_cities[p:p+l+1] = cities[p:p+l+1][::-1]
        return new_cities
    
#     @autojit
    def heaviside(self, argument, default):
        if argument < default:
            return 1
        else:
            return 0
    
#     @autojit
    def iterate(self, cities, T):
        new_cities = self.otherroutes(cities)
        HSp = self.distance(new_cities)
        HS = self.distance(cities)
        dE = HSp - HS
        if dE < 0 or (self.boltz and self.heaviside(dE, T)):
            cities = new_cities
        return cities
    
#     @autojit
    def continue_loop(self):
        while True:
            self.count += 1
            yield self.count
    
    def on_key(self, event):
        key = event.key
        if key == "t":
            self.T += 10
        elif key == "r":
            self.T -= 10
        elif key == "b":
            self.boltz = not self.boltz
    
#     @autojit
    def update(self, continue_loop):
        self.temp.append(self.T)
        cities = self.iterate(self.cities,self.T)
        self.distances.append(self.distance(cities))
        if continue_loop%10 == 0:
            self.T *= 0.9
        x = np.array(list(cities[:,1]) + list(cities[-1]))
        y = np.array(list(cities[:,0]) + list(cities[-1]))
        self.line.set_xdata(x)
        self.line.set_ydata(y)
        self.cities = cities
        self.clist.append(self.count)
        self.dlist.append(self.distance(self.cities))
        self.line1.set_xdata(self.clist)
        self.line1.set_ydata(self.dlist)
        self.line2.set_xdata(self.clist)
        self.line2.set_ydata(self.temp)
        if self.boltz:
            self.mode = "Boltzmann"
        else:
            self.mode = "threshold"
        self.data_text.set_text(self.data_temp %(self.count, self.T, self.distance(self.cities), self.mode))
        self.ax2.relim(True)
        self.ax3.relim(True)
        self.ax2.autoscale_view(True, True, True)
        self.ax3.autoscale_view(True, True, True)
        return self.line, self.data_text
    
    def run(self):
        cities = self.cities
        fig, (ax,ax2) = mp.subplots(2,1, gridspec_kw={"height_ratios":[10,4]}, facecolor="k")
        ax3 = ax2.twinx()
        
        ax.axis("off")
        ax.grid(False)
        x = np.array(list(cities[:,1]) + list(cities[-1]))
        y = np.array(list(cities[:,0]) + list(cities[-1]))
        self.line, = ax.plot(x,y, "wo-")
        ax.set_title("Simulated annealing")
        
        ax2.grid(True)
        ax2.set_xlabel("Iterations")
        ax2.set_ylabel("Average path length")
        ax3.set_ylabel("Temperature")
        ax3.grid(False)
        self.clist.append(self.count)
        self.dlist.append(self.distance(self.cities))
        self.temp.append(self.T)
        self.line1, = ax2.plot(self.clist, self.dlist, "w-")
        self.line2, = ax3.plot(self.clist, self.temp, "r-")
        self.ax2 = ax2; self.ax3 = ax3
        self.data_temp = r"\noindent iteration %i \\ temperature $T = %.2f$ \\ average path length $= %.2f$ \\ mode = %s"
        self.data_text = ax2.text(0.7, 0.65, "", transform=ax2.transAxes, color="c", fontsize=12)
        
        fig.canvas.mpl_connect("key_press_event", self.on_key)
        ani = anim.FuncAnimation(fig, self.update, self.continue_loop, interval=10, repeat=0)
        #mp.tight_layout()
        mp.show(block=True)

In [6]:
sim = TSP(20, boltz=False)
sim.run()

#### TSP algorithm with cpu multithreading

In [3]:
class TSP(object):
    
    def __init__(self, N, maxIters):
        rd.seed(420)
        self.T0 = N/8
        self.N = N
        self.cities = rd.random((N,2))
        self.distances = []
        self.temp = np.zeros(maxIters)*np.nan
        self.T = self.T0
        self.count = 0
        self.maxIters = maxIters
        self.clist = np.zeros(maxIters)*np.nan
        self.dlist = np.zeros(maxIters)*np.nan
        
#     @autojit
    def distance(self, cities):
        d = 0
        city1, city2 = cities, np.roll(cities, -1, axis=0)
        x1,y1 = city1[:,0], city1[:,1]
        x2,y2 = city2[:,0], city2[:,1]
        d = np.hypot((x2-x1), (y2-y1))
        return np.sum(d)
    
#     @autojit
    def get_dist(self):
        return np.sum(np.diff(self.cities))
    
#     @autojit
    def otherroutes(self, cities):
        new_cities = np.copy(cities)
        p = rd.randint(self.N)
        l = rd.randint(self.N//2)
        new_cities[p:p+l+1] = cities[p:p+l+1][::-1]
        return new_cities
    
#     @autojit
    def heaviside(self, argument, default):
        if argument < default:
            return 1
        else:
            return 0
    
#     @autojit
    def iterate(self, cities, T):
        new_cities = self.otherroutes(cities)
        HSp = self.distance(new_cities)
        HS = self.distance(cities)
        dE = HSp - HS
        if dE < 0 or self.heaviside(dE, T):
            cities = new_cities
        return cities
    
#     @autojit
    def continue_loop(self):
        while True:
            self.count += 1
            yield self.count
    
    def run(self):
        cities = self.cities
        fig, (ax,ax2) = mp.subplots(2,1, gridspec_kw={"height_ratios":[10,4]}, facecolor="k")
        
        ax.axis("off")
        ax.grid(False)
        x = np.array(list(cities[:,1]) + list(cities[-1]))
        y = np.array(list(cities[:,0]) + list(cities[-1]))
        self.line, = ax.plot(x,y, "wo-")
        ax.set_title("Simulated annealing")
        ax3 = ax2.twinx()
        
        ax2.grid(True)
        ax2.set_xlabel("Iterations")
        ax2.set_ylabel("Average path length")
        ax3.set_ylabel("Temperature")
        self.clist[0] = (self.count)
        self.dlist[0] = (self.distance(self.cities))
        self.temp[0] = self.T0
        self.line1, = ax2.plot(self.clist, self.dlist, "w-")
        self.line2, = ax3.plot(self.clist, self.temp, "r-")
        self.ax2 = ax2
        self.data_temp = r"\noindent iteration %i \\ temperature $T = %.2f$ \\ average path length $= %.2f$"
        self.data_text = ax2.text(0.7, 0.7, "", transform=ax2.transAxes, color="w", fontsize=14)
        
        with job.parallel_backend("threading", n_jobs=3):
            def init():
                self.line.set_ydata([np.nan]*len(self.cities))
                self.line1.set_ydata([np.nan]*len(self.dlist))
                self.line2.set_ydata([np.nan]*len(self.temp))
                self.data_text.set_text("")
                return self.line, self.line1, self.line2, self.data_text
        
            def update(i):
                cities = self.iterate(self.cities,self.T)
                self.distances.append(self.distance(cities))
                if i%10 == 0:
                    self.T *= 0.9
                x = np.array(list(cities[:,1]) + list(cities[-1]))
                y = np.array(list(cities[:,0]) + list(cities[-1]))
                self.line.set_xdata(x)
                self.line.set_ydata(y)
                self.cities = cities
                self.clist[i] = (self.count)
                self.dlist[i] = (self.distance(self.cities))
                self.temp[i] = (self.T)
                self.line1.set_xdata(self.clist)
                self.line1.set_ydata(self.dlist)
                self.line2.set_xdata(self.clist)
                self.line2.set_ydata(self.temp)
                self.data_text.set_text(self.data_temp %(self.count, self.T, self.distance(self.cities)))
                self.ax2.relim(True)
                self.ax2.autoscale_view(True, True, True)
                self.ax3.relim(True)
                self.count += 1
                return self.line, self.line1, self.line2, self.data_text
        
            ani = anim.FuncAnimation(fig, update, np.arange(1,self.maxIters), interval=30, blit=True, init_func=init)
        #mp.tight_layout()
        #mp.show(block=True)
            ani.save("tsp.mp4", writer="ffmpeg", savefig_kwargs={"facecolor":"black"})
        mp.show()

In [4]:
sim = TSP(30, maxIters=int(1e6))
sim.run()

AttributeError: 'TSP' object has no attribute 'ax3'